In [1]:
import os
import sys
import Utils
import numpy as np
import matplotlib.pylab as plt
import numpy as np
import sugartensor as tf # Please use pip install for this small package
# import tensorflow as tf
import os
import tqdm
from mpl_toolkits.mplot3d import Axes3D
colorsMap='jet'
import matplotlib
import matplotlib.cm as cmx
import glob
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"
tf.get_logger().setLevel('INFO')

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/donglin/anaconda3/envs/te

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dong

In [2]:
################################################################
train_num=10000
batSize=16
maxStep=50000 # fixed with learningRate and learningRateDecay
learningRate=0.001
learningRateDecay=0.995
lrType=1 #learning rate type
lr2Period=5
adam_beta1=0.9 # check adam optimization
adam_beta2=0.99
conWeightVar=['NNReg'] # variables to be loaded
saveStep=200 # frequency to save weight
maxKeepWeights=2000 # how many records to save (for disk)
stepsContinue=100000  # from which steps continu.
# conWeightPath="./NIPS-partseg-v1-temp10-chair/Exp4_maxStep_100000/"
conWeightPath=""
#For Debug and results printing
keepProb=0.99999
# Totally dosen't work if put dropout after max-pool layer. 
printStep=500

clas="mug-seg"
s1=2048
tfname="./mug_N2048.tfrecords"
dat="LX-run1"
lambda_ratio = 100.0 # init weight for chamfer distance 
beta_ratio = 0.1 # weight for correlation loss

dirSave="./{}_trNum_{}_maxStep_{}_lambda_{}".format(clas,
    dat,train_num, maxStep,lambda_ratio)
##################################################################

In [3]:
y,l,x,xl=Utils.read_from_tfrecords(tfname,
                                 ["D","P","T","TP"], batSize, [[s1,3],[s1],[s1,3],[s1]])
print(y.shape, l.shape, x.shape, xl.shape)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
Queue-base

In [4]:
def pairwise_dist(xt,y_p):
    a=xt.shape[1]
    b=y_p.shape[1]
    dist=tf.tile(tf.expand_dims(y_p,1),[1,a,1,1])-tf.tile(tf.expand_dims(xt,2),[1,1,b,1])
    dist=(dist[:,:,:,0]**2+dist[:,:,:,1]**2)
    return dist

def chamfer_loss(A,B):    
    r=tf.reduce_sum(A*A,2)
    r=tf.reshape(r,[int(r.shape[0]),int(r.shape[1]),1])
    r2=tf.reduce_sum(B*B,2)
    r2=tf.reshape(r2,[int(r.shape[0]),int(r.shape[1]),1])
    t=(r-2*tf.matmul(A, tf.transpose(B,perm=[0, 2, 1])) + tf.transpose(r2,perm=[0, 2, 1]))
    return tf.reduce_mean((tf.reduce_min(t, axis=1)+tf.reduce_min(t,axis=2))/2.0)

def Net(aa, yt, x):
    s=aa.shape[1]
    with tf.sg_context(name='NNReg', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        yt=tf.expand_dims(yt,2)
        
        v1=tf.expand_dims(x,2).sg_conv(dim=16, size=(1,1),  name='gen9',pad="SAME",bn=True)
        print('v1', v1.shape)        
        v2=v1.sg_conv(dim=64, size=(1,1), name='gen1',pad="SAME",bn=True)
        print('v2', v2.shape)        
        v3=v2.sg_conv(dim=128, size=(1,1), name='gen2',pad="SAME",bn=True)
        print('v3', v3.shape) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='gen3',pad="SAME",bn=True)
        print('v4', v4.shape) 
        v5=v4.sg_conv(dim=1024, size=(1,1), name='gen4',pad="SAME",bn=True)
        print('v5', v5.shape) 
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,s,1,1])
        print('v5_tiled', v5.shape)
        vv5=v5
        
        v1=yt.sg_conv(dim=16, size=(1,1),  name='gen99',pad="SAME",bn=True)
        print('v5', v1.shape)        
        v2=v1.sg_conv(dim=64, size=(1,1),  name='gen11',pad="SAME",bn=True)
        print('v5', v2.shape)
        v3=v2.sg_conv(dim=128, size=(1,1),  name='gen22',pad="SAME",bn=True)
        print('v5', v3.shape)
        v4=v3.sg_conv(dim=512, size=(1,1),  name='gen33',pad="SAME",bn=True)
        print('v5', v4.shape)
        v5=v4.sg_conv(dim=1024, size=(1,1),  name='gen44',pad="SAME",bn=True)
        print('v5', v5.shape)
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,s,1,1])
        print('v5tile', v5.shape)
        ff=tf.concat([tf.expand_dims(aa,2),v5], axis=-1)
        print(ff)
        ff=tf.concat([ff,vv5], axis=-1)
        print('ff2', ff)
        f1=ff.sg_conv(dim=256, size=(1,1),  name='f1',pad="SAME",bn=True)
        print('f1', f1)  
        f2=f1.sg_conv(dim=128, size=(1,1),  name='f2',pad="SAME",bn=True)  
        print('f2', f2)
        f3=f2.sg_conv(dim=3, size=(1,1),  name='f3',pad="SAME",bn=False, act="linear")
        print('f3', f3)  
        f3=tf.squeeze(f3,axis=2)
        print('f3_out', f3)
        
    return f3

def part_set(y):#used in training, deformed shape
    with tf.sg_context(name='RegSeg1', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        v1=tf.expand_dims(y,2).sg_conv(dim=16, size=(1,1),  name='gen9',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=64, size=(1,1), name='gen1',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=128, size=(1,1), name='gen2',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='gen3',pad="SAME",bn=True) 
        print('v4', v4)
        v5=v4.sg_conv(dim=1024, size=(1,1), name='gen4',pad="SAME",bn=True) 
        print('v5b', v5)
        v5 = tf.reduce_max(v5, axis=1)
        print('v5r', v5)
        v5 = tf.expand_dims(v5,axis=1)
        print('v5e', v5)
        v5=tf.tile(v5,[1,int(y.shape[1]),1,1]) #[B,N,1,1024]
        print('v5a', v5)
        
        ff=tf.concat([tf.expand_dims(y,2),v5], axis=-1) #[B,N,1,3+1024]
        print('ff', ff)
        f1=ff.sg_conv(dim=256, size=(1,1),  name='f1',pad="SAME",bn=True)
        print(f1)  
        f1=tf.concat([tf.expand_dims(y,2),f1], axis=-1) 
        print(f1)
        f2=f1.sg_conv(dim=128, size=(1,1),  name='f2',pad="SAME",bn=True) 
        print(f2) 
        f2=tf.concat([tf.expand_dims(y,2),f2], axis=-1) 
        print(f2)
        feat=f2.sg_conv(dim=128, size=(1,1),  name='feat',pad="SAME",bn=True)
        print(feat)
        f3=feat.sg_conv(dim=4, size=(1,1),  name='f3',pad="SAME",bn=False, act="linear") 
        print(f3)
        f3=tf.squeeze(f3,axis=2) #[B,N,4]
        print(f3)
        feat = tf.squeeze(feat,axis=2)
        print(feat)
    return f3, feat


In [5]:
net_out = Net(x,x,y) + x
print(net_out.shape)
print("=============================================")
out1, out2 = part_set(net_out)

print(out1.shape, out2.shape)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
v1 (16, 2048, 1, 16)
v2 (16, 2048, 1, 64)
v3 (16, 2048, 1, 128)
v4 (16, 2048, 1, 512)
v5 (16, 2048, 1, 1024)
v5_tiled (16, 2048, 1, 1024)
v5 (16, 2048, 1, 16)
v5 (16, 2048, 1, 64)
v5 (16, 2048, 1, 128)
v5 (16, 2048, 1, 512)
v5 (16, 2048, 1, 1024)
v5tile (16, 2048, 1, 1024)
Tensor("NNReg/concat:0", shape=(16, 2048, 1, 1027), dtype=float32)
ff2 Tensor("NNReg/concat_1:0", shape=(16, 2048, 1, 2051), dtype=float32)
f1 Tensor("NNReg/f1/out:0", shape=(16, 2048, 1, 256), dtype=float32)
f2 Tensor("NNReg/f2/out:0", shape=(16, 2048, 1, 128), dtype=float32)
f3 Tensor("NNReg/f3/out:0", shape=(16, 2048, 1, 3), dtype=float32)
f3_out Tensor("NNReg/Squeeze:0", shape=(16, 2048, 3), dtype=float32)
(16, 2048, 3)
v4 Tensor("RegSeg1/gen3/out:0", shape=(16, 2048, 1, 512), dtype=float32)
v5b Tensor("RegSeg1/gen4/out:0", shape=(16, 2048, 1, 1024), dtype=float32)
v5r Tensor("RegSeg1/Max:0", shape=(16, 1, 1024), dtype=f

In [7]:
global_step = tf.Variable(1, trainable=False,name='global_step')
tf.train.exponential_decay(lambda_ratio, global_step,
                                                  batSize, 0.999, staircase=False)

TensorShape([])

In [1]:
############################
#Author
#Lingjing Wang and Xiang Li
#Date: June 2020
############################

import os
import sys
import Utils
import numpy as np
import matplotlib.pylab as plt
import numpy as np
import sugartensor as tf # Please use pip install for this small package
# import tensorflow as tf
import os
import tqdm
from mpl_toolkits.mplot3d import Axes3D
colorsMap='jet'
import matplotlib
import matplotlib.cm as cmx
import glob
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"

def pairwise_dist(xt,y_p):
    a=xt.shape[1]
    b=y_p.shape[1]
    dist=tf.tile(tf.expand_dims(y_p,1),[1,a,1,1])-tf.tile(tf.expand_dims(xt,2),[1,1,b,1])
    dist=(dist[:,:,:,0]**2+dist[:,:,:,1]**2)
    return dist

def chamfer_loss(A,B):    
    r=tf.reduce_sum(A*A,2)
    r=tf.reshape(r,[int(r.shape[0]),int(r.shape[1]),1])
    r2=tf.reduce_sum(B*B,2)
    r2=tf.reshape(r2,[int(r.shape[0]),int(r.shape[1]),1])
    t=(r-2*tf.matmul(A, tf.transpose(B,perm=[0, 2, 1])) + tf.transpose(r2,perm=[0, 2, 1]))
    return tf.reduce_mean((tf.reduce_min(t, axis=1)+tf.reduce_min(t,axis=2))/2.0)

def Net(aa, yt, x):
    s=aa.shape[1]
    with tf.sg_context(name='NNReg', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        yt=tf.expand_dims(yt,2)
        
        v1=tf.expand_dims(x,2).sg_conv(dim=16, size=(1,1),  name='gen9',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=64, size=(1,1), name='gen1',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=128, size=(1,1), name='gen2',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='gen3',pad="SAME",bn=True) 
        v5=v4.sg_conv(dim=1024, size=(1,1), name='gen4',pad="SAME",bn=True) 
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,s,1,1])
        vv5=v5
        
        v1=yt.sg_conv(dim=16, size=(1,1),  name='gen99',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=64, size=(1,1),  name='gen11',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=128, size=(1,1),  name='gen22',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1),  name='gen33',pad="SAME",bn=True) 
        v5=v4.sg_conv(dim=1024, size=(1,1),  name='gen44',pad="SAME",bn=True) 
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,s,1,1])
        
        ff=tf.concat([tf.expand_dims(aa,2),v5], axis=-1) 
        ff=tf.concat([ff,vv5], axis=-1) 
        f1=ff.sg_conv(dim=256, size=(1,1),  name='f1',pad="SAME",bn=True)  
        f2=f1.sg_conv(dim=128, size=(1,1),  name='f2',pad="SAME",bn=True)  
        
        f3=f2.sg_conv(dim=3, size=(1,1),  name='f3',pad="SAME",bn=False, act="linear")  
        f3=tf.squeeze(f3,axis=2)
        
    return f3

def part_set(y):#used in training, deformed shape
    with tf.sg_context(name='RegSeg1', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        v1=tf.expand_dims(y,2).sg_conv(dim=16, size=(1,1),  name='gen9',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=64, size=(1,1), name='gen1',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=128, size=(1,1), name='gen2',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='gen3',pad="SAME",bn=True) 
        v5=v4.sg_conv(dim=1024, size=(1,1), name='gen4',pad="SAME",bn=True) 
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,int(y.shape[1]),1,1]) #[B,N,1,1024]
        
        ff=tf.concat([tf.expand_dims(y,2),v5], axis=-1) #[B,N,1,3+1024]
        f1=ff.sg_conv(dim=256, size=(1,1),  name='f1',pad="SAME",bn=True)  
        f1=tf.concat([tf.expand_dims(y,2),f1], axis=-1) 
        f2=f1.sg_conv(dim=128, size=(1,1),  name='f2',pad="SAME",bn=True)  
        f2=tf.concat([tf.expand_dims(y,2),f2], axis=-1) 
        feat=f2.sg_conv(dim=128, size=(1,1),  name='feat',pad="SAME",bn=True)
        f3=feat.sg_conv(dim=4, size=(1,1),  name='f3',pad="SAME",bn=False, act="linear") 
        f3=tf.squeeze(f3,axis=2) #[B,N,4]
        feat = tf.squeeze(feat,axis=2)
    return f3, feat

def part_set_pred(yp, y): #used in testing, input shape, deformed shape
    with tf.sg_context(name='RegSeg1', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        
        v1=tf.expand_dims(y,2).sg_conv(dim=16, size=(1,1),  name='gen9',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=64, size=(1,1), name='gen1',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=128, size=(1,1), name='gen2',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='gen3',pad="SAME",bn=True) 
        v5=v4.sg_conv(dim=1024, size=(1,1), name='gen4',pad="SAME",bn=True) 
        v5=tf.tile(tf.expand_dims(tf.reduce_max(v5, axis=1),axis=1),[1,int(y.shape[1]),1,1])
        
        ff=tf.concat([tf.expand_dims(yp,2),v5], axis=-1) 
        f1=ff.sg_conv(dim=256, size=(1,1),  name='f1',pad="SAME",bn=True)  
        f1=tf.concat([tf.expand_dims(yp,2),f1], axis=-1) 
        f2=f1.sg_conv(dim=128, size=(1,1),  name='f2',pad="SAME",bn=True)  
        f2=tf.concat([tf.expand_dims(yp,2),f2], axis=-1) 
        feat=f2.sg_conv(dim=128, size=(1,1),  name='feat',pad="SAME",bn=True)
        f3=feat.sg_conv(dim=4, size=(1,1),  name='f3',pad="SAME",bn=False, act="linear") 
        f3=tf.squeeze(f3,axis=2)
        feat = tf.squeeze(feat,axis=2)
    return f3

def part_encoder(feat): #used in testing, input shape, deformed shape
    with tf.sg_context(name='part_encoder', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        
        v1=tf.expand_dims(feat,2).sg_conv(dim=128, size=(1,1),  name='part_en1',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=128, size=(1,1), name='part_en2',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=256, size=(1,1), name='part_en3',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=1024, size=(1,1), name='part_en4',pad="SAME",bn=True) 
        v4=tf.expand_dims(tf.reduce_max(v4, axis=1),axis=1) #[B,1,1,1024]
        v4=tf.squeeze(v4,axis=1)
        v4=tf.squeeze(v4,axis=1) #[B,1024]
    return v4

def part_encoder_v2(feat): #used in testing, input shape, deformed shape
    with tf.sg_context(name='part_encoder', stride=1, act='leaky_relu', bn=True, reuse=tf.AUTO_REUSE): 
        v1 = tf.expand_dims(feat,0)
        v1 = tf.expand_dims(v1,2)
        v1 = v1.sg_conv(dim=128, size=(1,1),  name='part_en1',pad="SAME",bn=True)        
        v2=v1.sg_conv(dim=128, size=(1,1), name='part_en2',pad="SAME",bn=True)        
        v3=v2.sg_conv(dim=256, size=(1,1), name='part_en3',pad="SAME",bn=True) 
        v4=v3.sg_conv(dim=512, size=(1,1), name='part_en4',pad="SAME",bn=True) 
        v4=tf.expand_dims(tf.reduce_max(v4, axis=1),axis=1) #[B,1,1,1024]
        v4=tf.squeeze(v4,axis=1)
        v4=tf.squeeze(v4) #[1,1024]
    return v4

def PearsonCoeff(x,y):
    vx = x - tf.reduce_sum(x)
    vy = y - tf.reduce_sum(y)

    cost = tf.reduce_sum(vx * vy) / (tf.sqrt(tf.reduce_sum(vx ** 2)) * tf.sqrt(tf.reduce_sum(vy ** 2)))
    return cost

def train():
#     tf.reset_default_graph()
    tf.set_random_seed(888)
    print("*****************************************")
    print("Training started with random seed: {}".format(111))
    print("Batch started with random seed: {}".format(111))
    
    #read data
    y,l,x,xl=Utils.read_from_tfrecords(tfname,
                                 ["D","P","T","TP"], batSize, [[s1,3],[s1],[s1,3],[s1]])
    global_step = tf.Variable(1, trainable=False,name='global_step')
    yp=Net(x,x,y)+x #deformed shape
    
    #part seg during training
    plogit, part_feat=part_set(yp) #[B,N,4], [B,N,64]
    pred_cls = tf.argmax(plogit, axis=-1)
    
    #prediction during testing
    pl = tf.argmax(part_set_pred(y,yp), axis=-1)
    # pl=tf.argmax(part_set(yp), axis=-1)
    
    cross_ent_ls=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=plogit, labels=tf.cast(xl,dtype=tf.int32)))
    chamf_ls=chamfer_loss(yp,y)  
    print('chamf_ls, cross_ent_ls', chamf_ls, cross_ent_ls)
    
    lambda_ratio_tf = tf.train.exponential_decay(lambda_ratio, global_step,
                                                  batSize, 0.999, staircase=False) 
    Loss =  cross_ent_ls + chamf_ls*lambda_ratio_tf
    
    #correlation loss 
    print('part_feat.shape: ',part_feat)
    
    M_list = []
    E_same_list = []
    E_diff_list = []
    
    for c_id in range(4):
        M_c = []
        for s_id in range(batSize):
            pred_cls_s = pred_cls[s_id] #[N,4]
            mask0 = tf.boolean_mask(part_feat[s_id], tf.equal(pred_cls_s,c_id))
#             def f1(): return part_encoder_v2(mask0) #[1,512]
            def f1(): return tf.reduce_max(mask0, axis=0) #[1,128]
            def f2(): return tf.ones([128,])
            mask0 = tf.cond(tf.reduce_any(tf.equal(pred_cls_s,c_id)), f1, f2)
            M_c.append(mask0)
            
        M_c = tf.stack(M_c) #[B,128]
        M_list.append(tf.reduce_mean(M_c, axis=0))
        corr_same_list = []
        for ii in range(batSize):
            for jj in range(ii+1, batSize):
                corr_same_list.append(PearsonCoeff(M_c[ii], M_c[jj]))
        E_same = tf.reduce_min(corr_same_list)
        E_same_list.append(E_same)
    
    for i in range(4):
        for j in range(i+1,4):
            pearson_r = PearsonCoeff(M_list[i], M_list[j])
            E_diff_list.append(pearson_r)
    
    E_corr = -tf.reduce_mean(E_same_list) + tf.reduce_max(E_diff_list)
    
#     Loss = Loss + beta_ratio*E_corr
    
    def f1(): return Loss + E_corr* beta_ratio
    def f2(): return Loss
    
    Loss = tf.cond(global_step>5000, f1, f2) #从5000step开始使用correlation loss

    #Learning Rate****************************************************************************
    lr = tf.train.exponential_decay(learningRate, global_step,
                                                  batSize, learningRateDecay, staircase=False) 
    # Optimization Algo************************************************************************
    train_step = tf.train.AdamOptimizer(learning_rate=lr,
                                                    beta1=adam_beta1,
                                                    beta2=adam_beta2
                                                   ).minimize(Loss,global_step=global_step,var_list=[i for i in 
                                                                                                     tf.trainable_variables()])
    
    saver = tf.train.Saver(max_to_keep=int(maxKeepWeights))
    init_op = tf.group(tf.global_variables_initializer(),
                               tf.local_variables_initializer())
    
    # Continue Training************************************************************************
    if len(conWeightPath)>0:
        print("Continue Training...")
        tmp_var_list={}
        if len(conWeightVar)==0:
            print("For all variables")
            globals()['conWeightVar']={''}
        else:
            print("Training variables: {}".format(conWeightVar))
            
        for j in conWeightVar: 
            for i in tf.global_variables():
                if i.name.startswith(j):
                    tmp_var_list[i.name[:-2]] = i      
                    
        saver1=tf.train.Saver(tmp_var_list)     
    
    num_params = np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])
    print('num_params', num_params)
    
    
    # Training**********************************************************************************    
    with tf.Session() as sess:
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # Read Weight******************************
        if len(conWeightPath)>0:
            print(conWeightPath)
            if stepsContinue==-1:            
                STEPS=sorted([int(i.split("/")[-1].split(".")[1].split("-")[-1]) for i in glob.glob(conWeightPath+"/*meta")])
                print("hahaha",STEPS)
                globals()['stepsContinue']=STEPS[-1]
                
            wtt=glob.glob(conWeightPath+"/*{}*meta".format(stepsContinue))[0][:-5]
            print("Reading Weight:{}".format(wtt))
            saver1.restore(sess,wtt)
            print('Weight is successfully updated from: {}'.format(wtt))  
        #*******************************************    
        stepst = sess.run(global_step)
        for t in tqdm.tqdm(range(stepst,int(maxStep)+1)):      
            _= sess.run([train_step]) 
            if t % saveStep==0:
                if not os.path.exists(dirSave):
                    os.makedirs(dirSave)
                saver.save(sess, dirSave + '/model.ckpt', global_step=t)
            
            if t%200==0:
                print('test', sess.run([tf.reduce_mean(E_same_list), tf.reduce_max(E_diff_list)]))
                
            if t%100==0:
                chamf_ls_val, lambda_ratio_val, cross_ent_ls_val, total_ls_val=sess.run([chamf_ls, lambda_ratio_tf, cross_ent_ls, Loss])
                print('chamf_ls: {:.5f}, cd weight: {:.5f}, cross_ent_ls: {:.5f}, correlation loss: {:.5f}, total_loss: {:.5f}'.format(chamf_ls_val, lambda_ratio_val, cross_ent_ls_val, total_ls_val-chamf_ls_val-cross_ent_ls_val, total_ls_val))
                
#             if t % printStep==0:
#                 yyp,yy,ll,ppl,xx,xxl=sess.run([yp,y,l,pl,x,xl])  #deformed shape, input shape, input label, prediction label, template shape, template label
#                 plot_result(yyp,yy,ll,ppl,xx,xxl)
                
        coord.request_stop()
        coord.join(threads)   
        

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/donglin/anaconda3/envs/te

/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donglin/anaconda3/envs/temp-seg37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dong

In [2]:
################################################################
train_num=10000
batSize=16
maxStep=50000 # fixed with learningRate and learningRateDecay
learningRate=0.001
learningRateDecay=0.995
lrType=1 #learning rate type
lr2Period=5
adam_beta1=0.9 # check adam optimization
adam_beta2=0.99
conWeightVar=['NNReg'] # variables to be loaded
saveStep=200 # frequency to save weight
maxKeepWeights=2000 # how many records to save (for disk)
stepsContinue=100000  # from which steps continu.
# conWeightPath="./NIPS-partseg-v1-temp10-chair/Exp4_maxStep_100000/"
conWeightPath=""
#For Debug and results printing
keepProb=0.99999
# Totally dosen't work if put dropout after max-pool layer. 
printStep=500

clas="mug-seg"
s1=2048
tfname="./mug_N2048.tfrecords"
dat="LX-run1"
lambda_ratio = 100.0 # init weight for chamfer distance 
beta_ratio = 0.1 # weight for correlation loss

dirSave="./{}_trNum_{}_maxStep_{}_lambda_{}".format(clas,
    dat,train_num, maxStep,lambda_ratio)
##################################################################

In [3]:
train()

*****************************************
Training started with random seed: 111
Batch started with random seed: 111
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based 

2022-03-14 19:39:43.085045: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2022-03-14 19:39:43.104980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-14 19:39:43.105068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:2b:00.0
2022-03-14 19:39:43.105109: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Could not dlopen library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2022-03-14 19:39:43.105139: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Could not dlopen library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file:

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


2022-03-14 19:39:44.951913: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
  0%|          | 20/50000 [01:38<68:18:39,  4.92s/it]

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[{{node input_producer/input_producer_EnqueueMany}}]]



2022-03-14 19:41:24.863421: W tensorflow/core/kernels/queue_base.cc:277] _1_shuffle_batch/random_shuffle_queue: Skipping cancelled enqueue attempt with queue not closed
2022-03-14 19:41:24.863469: W tensorflow/core/kernels/queue_base.cc:277] _0_input_producer: Skipping cancelled enqueue attempt with queue not closed


KeyboardInterrupt: 